In [8]:
import csv
import numpy

values = []

with open("dataset/ambient_temperature_system_failure.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for i, line in enumerate(reader):
        if not i:
            continue
        value = float(line[0].split(',')[1])
        value = (float(value) - 32) * 5 / 9

        values.append(value)

x, y = [], []
for i in range(48, len(values) - 24, 24):
    x.append(values[i - 48:i])
    y.append(values[i:i + 24])

x, y = numpy.array(x), numpy.array(y)

fit_start = int(len(x) * 0.2)
x_fit, y_fit = x[fit_start:], y[fit_start:]
x_valid, y_valid = x[:fit_start], y[:fit_start]

In [43]:
import sklearn.model_selection
import sklearn.svm
import sklearn.multioutput
import sklearn.metrics

def score(*args, **kwargs):
    return sklearn.metrics.mean_squared_error(*args, squared=False, **kwargs)


search = sklearn.model_selection.GridSearchCV(
    sklearn.multioutput.MultiOutputRegressor(sklearn.svm.SVR()),
    {'estimator__C': [6, 7, 8]},
    scoring=sklearn.metrics.make_scorer(score, greater_is_better=False))

search.fit(x_fit, y_fit)

GridSearchCV(estimator=MultiOutputRegressor(estimator=SVR()),
             param_grid={'estimator__C': [6, 7, 8]},
             scoring=make_scorer(score, greater_is_better=False))

In [44]:
search.cv_results_

{'mean_fit_time': array([0.07095089, 0.06982546, 0.07134981]),
 'std_fit_time': array([0.0079291 , 0.00245155, 0.00646164]),
 'mean_score_time': array([0.01377077, 0.01417546, 0.01358666]),
 'std_score_time': array([0.00047489, 0.00053198, 0.00094877]),
 'param_estimator__C': masked_array(data=[6, 7, 8],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'estimator__C': 6}, {'estimator__C': 7}, {'estimator__C': 8}],
 'split0_test_score': array([-0.85498773, -0.85945549, -0.86226141]),
 'split1_test_score': array([-1.29125325, -1.27631318, -1.26634836]),
 'split2_test_score': array([-0.64326025, -0.64460761, -0.64724773]),
 'split3_test_score': array([-1.20473515, -1.20882402, -1.21358101]),
 'split4_test_score': array([-2.13002372, -2.13233368, -2.13713761]),
 'mean_test_score': array([-1.22485202, -1.2243068 , -1.22531523]),
 'std_test_score': array([0.50974468, 0.50923959, 0.50974731]),
 'rank_test_score': array([2, 1, 3], dtype

In [45]:
search.best_params_, search.best_score_

({'estimator__C': 7}, -1.2243067951746291)

In [46]:
score(search.best_estimator_.predict(x_valid), y_valid)

1.3016646670951195